In [1]:
import os
import shutil
import re

In [2]:
os.getcwd()

'/root/data/SE-VAE/jupyter'

In [15]:
def my_filter(path):
    log_path = os.path.join('..', path, 'log.out')
    log_s = open(log_path, 'r').readlines()
    test_path = os.path.join('..', path, 'test.out')
    test_s = open(log_path, 'r').readlines()
    #if False:
        #print(log_path)
        #return False
    #elif s[4] != 'history_length: 40\n' and s[5] != 'forward_length: 160\n':
        #print(log_path)
        #return False
    #elif 'Error' in test_s[-2]:
        #print(log_path)
        #return False
    #else:
    return True
    
def generating_dir(base_dir, root_dir):
    ret = []
    for file in os.listdir(os.path.join(root_dir, base_dir)):
        if file == 'tmp':
            continue
        path = os.path.join(base_dir, file)
        if os.path.isdir(os.path.join(root_dir, path)):
            ret = ret + generating_dir(path, root_dir)
        elif file == 'log.out':                                                   #and os.path.exists(os.path.join(root_dir, base_dir, 'best.pth')):
            if my_filter(os.path.split(path)[0]):
                ret.append(os.path.split(path)[0])
            else:
                continue
    return ret

path_list = generating_dir('ckpt'+os.path.sep, '..'+os.path.sep)
# print('\n'.join(path_list))
print('共%d个训练模型'%len(path_list))


共120个训练模型


In [16]:
#检查log文件#可不可以直接找不存在best.pth的路径
error_log_list=[]                    #错误log文件路径集合
for t in range(len(path_list)):      #第t个临时列表中的路径
    error_log = False
    #print('t=',t)
    if not os.path.exists(os.path.join('..', path_list[t], 'best.pth')):
        error_log = True                   #存在error的标志
        error_line = "无模型"
    else:
        f = open('..'+os.path.sep+path_list[t]+os.path.sep+'log.out','r')
        log_data = f.readlines()
        f.close()
        for line in log_data[-100:]:
            if re.search('Error',line):
                error_log, error_line = True, line
                break
            if re.search('Early stopping at epoch',line):
                error_log = False
                break
            if re.search('Training finished', line):
                error_log = False
                break
        else:
            if len(log_data)<1000:
                error_log, error_line = True, 'Not finished'
          
    if error_log==True:
        error_log_list.append(path_list[t])
        print('Error: ', path_list[t], error_line, '\n', 'Lines: %d' % len(log_data))

Error:  ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=1,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512,train/schedule=val_step/2022-05-04_07-49-29 KeyError: 0
 
 Lines: 883
Error:  ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=5,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512,train/schedule=val_step/2022-05-04_08-27-58 AssertionError
 
 Lines: 1565
Error:  ckpt/winding/ct_True/ode_rssm/ode_rssm_ct_time=True,model.iw_trajs=10,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512/2022-05-03_17-31-48 无模型 
 Lines: 1219
Error:  ckpt/winding/ct_True/ode_rssm/ode_rssm_ct_time=True,model.iw_trajs=5,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=or

In [17]:
def Delete_log(path):
    print("function have been triggered")
    print(path)
    shutil.rmtree(path)         #删除当前目录文件夹以及文件夹内的所有文件
    path=os.path.dirname(path)
    print(path)
    if not os.listdir(path):
        Delete_log(path)
    else:
        print("Delete accomplished")
        return 0
Delete_list=[]
if error_log_list:
    print("log文件报错路径如下：")
    for line in range(len(error_log_list)):
        Delete_list.append(error_log_list[line])
        print(Delete_list[line])
    ju=input("确定删除路径文件及其所在文件夹？ 请输入'yes' or 'no'")
    if ju=='yes':
        for line in range(len(Delete_list)):
            print(line)
            Delete_log('..'+os.path.sep+Delete_list[line])
    else:
        print("删除取消，保留路径文件")
else:
    print('未发现log文件报错')

log文件报错路径如下：
ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=1,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512,train/schedule=val_step/2022-05-04_07-49-29
ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=5,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512,train/schedule=val_step/2022-05-04_08-27-58
ckpt/winding/ct_True/ode_rssm/ode_rssm_ct_time=True,model.iw_trajs=10,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512/2022-05-03_17-31-48
ckpt/winding/ct_True/ode_rssm/ode_rssm_ct_time=True,model.iw_trajs=5,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512/2022-05-03_16-24-05


确定删除路径文件及其所在文件夹？ 请输入'yes' or 'no' yes


0
function have been triggered
../ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=1,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512,train/schedule=val_step/2022-05-04_07-49-29
../ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=1,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512,train/schedule=val_step
function have been triggered
../ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=1,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_seed=0,sp=0.5,train.batch_size=512,train/schedule=val_step
../ckpt/winding/ct_True/ode_rssm_schedule/ode_rssm_ct_time=True,model.iw_trajs=1,model.k_size=16,model.ode_ratio=all,model.ode_solver=rk4,model.ode_type=orth,model.state_size=16,random_see